## imports

In [2]:
import os
from geopandas import read_file
from geojson import Feature, FeatureCollection, dump
import json
import simplekml

from pyproj import transform, Proj
#projectionIn = Proj(init='epsg:32627')

data_partition = 'c:\\data\\ReSOW'
data_partition = 'd:\\sartwl\\uk\\scripts'

#grid_dir = os.path.join(data_partition, 'grids')
base_file_path = os.path.join(data_partition, 'coast.geojson')
#out_file_path = os.path.join(grid_dir, 'coast.geojson')

base_grid = read_file(base_file_path)

crs = {
        "type": "name",
        "properties": {"name": base_grid.crs['init']}
      }

def WKT(hexgrid_df):
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    print(f'POLYGON(({west:.3f} {north:.3f}, {west:.3f} {south:.3f} ,' + \
           f'{east:.3f} {south:.3f}, {east:.3f} {north:.3f}, {west:.3f} {north:.3f}))')
    
def CWWKT(hexgrid_df):

    proj_BGS = Proj(init="epsg:27700")
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    west_4326, north_4326 = proj_BGS(west, north, inverse=True)
    east_4326, south_4326 = proj_BGS(east, south, inverse=True)
    
    print(f'POLYGON(({west_4326:.3f} {north_4326:.3f}, {west_4326:.3f} {south_4326:.3f} ,' + \
           f'{east_4326:.3f} {south_4326:.3f}, {east_4326:.3f} {north_4326:.3f},' + \
          f' {west_4326:.3f} {north_4326:.3f}))')
    print()
    print(f'[[[{west_4326:.3f}, {north_4326:.3f}], ' + \
            f'[{west_4326:.3f}, {south_4326:.3f}], ' + \
            f'[{east_4326:.3f}, {south_4326:.3f}], ' + \
            f'[{east_4326:.3f}, {north_4326:.3f}], ' + \
            f'[{west_4326:.3f}, {north_4326:.3f}]]]')    

def eeROI(grid_df):

    proj_BGS = Proj(init="epsg:27700")
    
    west = grid_df['west'].values[0]
    east = grid_df['east'].values[0]
    north = grid_df['north'].values[0]
    south = grid_df['south'].values[0]
    
    west_4326, north_4326 = proj_BGS(west, north, inverse=True)
    east_4326, south_4326 = proj_BGS(east, south, inverse=True)
    
    print(f'[[[{west_4326:.3f}, {north_4326:.3f}], ' + \
            f'[{west_4326:.3f}, {south_4326:.3f}], ' + \
            f'[{east_4326:.3f}, {south_4326:.3f}], ' + \
            f'[{east_4326:.3f}, {north_4326:.3f}], ' + \
            f'[{west_4326:.3f}, {north_4326:.3f}]]]')  

## run the code

In [4]:
out_file_path = os.path.join(data_partition, 'gbr_coast.geojson')

features = []
for index, row in base_grid.iterrows():
    features.append(Feature(geometry=row['geometry'],
                            properties={'id': index}))

feature_collection = FeatureCollection(features, crs=crs)

with open(out_file_path, 'w') as f:
   dump(feature_collection, f)

In [ ]:
for cell in [7]:
    
    hexcell_file_path = os.path.join(grid_dir, f'grid_{str(cell)}.geojson')

    features = []
    for index, row in base_grid.iterrows():
        
        if index == cell:
            features.append(Feature(geometry=row['geometry'],
                            properties={'id': index,
#                                        'west': row['left'],
#                                        'east': row['right'],
#                                        'south': row['bottom'],
#                                        'north': row['top'],
                                       }))
#            print(row['geometry'])
            
    feature_collection = FeatureCollection(features, crs=crs)

    with open(hexcell_file_path, 'w') as f:
       dump(feature_collection, f)
    
#    eeROI(read_file(hexcell_file_path))
    print()

In [ ]:
with open(base_file_path) as f:
    data = json.load(f)


projectionIn = Proj(init='epsg:32627')

kml = simplekml.Kml()
for feature in data['features']:
    coord_tuples = []
    coord_list = (feature['geometry']['coordinates'][0][0])
    for coord in coord_list:
        long_w, lat_n = projectionIn(coord[0], coord[1], inverse=True)
#        long_w, lat_n = transform(projectionIn, projectionIn,
#                                  coord[0], coord[1])
        coord_tuples.append((long_w, lat_n))
    print(coord_tuples)
    name = str((int(feature['properties']['id'])))
    if feature['geometry']['type'] == 'MultiPolygon':
        kml.newpolygon(name=name,
                       description='test',
                       outerboundaryis=coord_tuples)
kml.save(base_file_path.replace('geojson', 'kml'))

In [ ]:
import rasterio
import pyproj

file = 'S2B_MSIL2A_20210903T144719_N0301_R139_T19LDG_20210903T184347.tif'
# reading the GeoTIFF file and returning the bounds and the coordinate reference
# system (CRS) which is EPS:32719 for Sentinel-2
with rasterio.open(file) as src:
    bounds = src.bounds
    crs_from = src.crs

# defining the CRS to be transformed into
epsg_id = 4326
crs_to = pyproj.CRS.from_epsg(epsg_id)

# defining the transformer which transforms from one CRS to another
proj = pyproj.Transformer.from_crs(crs_from=crs_from, crs_to=crs_to)
bounds_4326 = proj.transform_bounds(*bounds)

print('bounds in EPSG 32719:', bounds)
print('bounds in EPSG 4326:', bounds_4326)

In [1]:
import os
from geopandas import read_file
from geojson import Feature, FeatureCollection, dump
import json
from pyproj import transform, Proj
projection = Proj(init='epsg:27700')

data_partition = 'c:\\data\\ReSOW\\grids'

base_file_path = os.path.join(data_partition, 'coast_original.geojson')
out_file_path = os.path.join(data_partition, 'coast.geojson')

base_grid = read_file(base_file_path)

crs = {
        "type": "name",
        "properties": {"name": base_grid.crs['init']}
      }

def WKT(hexgrid_df):
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    print(f'POLYGON(({west} {north}, {west} {south} ,' + \
           f'{east} {south}, {east} {north}, {west} {north}))')

    
def CWWKT(hexgrid_df):

    proj_BGS = Proj(init="epsg:27700")
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    west_4326, north_4326 = proj_BGS(west, north, inverse=True)
    east_4326, south_4326 = proj_BGS(east, south, inverse=True)
    
    print(f'POLYGON(({west_4326} {north_4326}, {west_4326} {south_4326} ,' + \
           f'{east_4326} {south_4326},' + \
           f'{east_4326} {north_4326},' + \
           f' {west_4326} {north_4326}))')
    print('')
    print(f'({west_4326 + (west_4326-east_4326)/2:.3f}, ' + \
          f'{south_4326 + (north_4326-south_4326)/2:.3f})')

          
def eeGeoJson(hexgrid_df):
          
    proj_BGS = Proj(init="epsg:27700")
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    west_4326, north_4326 = proj_BGS(west, north, inverse=True)
    east_4326, south_4326 = proj_BGS(east, south, inverse=True)
          
    eeGJ = { "type": "Polygon",
            "coordinates": [[[west_4326, north_4326],
                             [west_4326, south_4326],
                             [east_4326, south_4326],
                             [east_4326, north_4326]]]
           }
    print(eeGJ)

In [2]:
features = []
for index, row in base_grid.iterrows():
    features.append(Feature(geometry=row['geometry'],
                            properties={'id': index}))

feature_collection = FeatureCollection(features, crs=crs)

with open(out_file_path, 'w') as f:
   dump(feature_collection, f)

print('done')

done


In [ ]:
for cell in [7, 16, 17, 32, 33, 46, 57,58, 72,73, 79, 80, 88, 89, 93]:
    
    hexcell_file_path = os.path.join(data_partition, f'grid_{str(cell)}.geojson')

    features = []
    for index, row in base_grid.iterrows():
        
        if cell == row['id']:
            features.append(Feature(geometry=row['geometry'],
                            properties={'id': cell,
#                                        'west': row['left'],
#                                        'east': row['right'],
#                                        'south': row['bottom'],
#                                        'north': row['top'],
                                       }))
            print(row['geometry'])
            
    feature_collection = FeatureCollection(features, crs=crs)

    with open(hexcell_file_path, 'w') as f:
       dump(feature_collection, f)
    print(hexcell_file_path)
    
#    CWWKT(read_file(hexcell_file_path))
    print()

In [ ]:
var geojsonObject = {
  "type": "Polygon",
  "coordinates": [
    [
      [
        -122.085,
        37.423
      ],
      [
        -122.092,
        37.424
      ],
      [
        -122.085,
        37.418
      ],
      [
        -122.085,
        37.423
      ]
    ]
  ]
};

((( ,  380000, 270000 330000, 220000 , 220000 380000)))

var xmin = 270000;
var ymin = 330000;
var xmax = 220000;
var ymax = 380000;

var bl = ee.Geometry.Point([xmin, ymin]);
var br = ee.Geometry.Point([xmax, ymin]);
var tl = ee.Geometry.Point([xmin, ymax]);
var tr = ee.Geometry.Point([xmax, ymax]);

var cali_albers = ee.Projection('EPSG:3310')

var bbox_coords = ee.List([bl, br, tr, tl, bl])
var bbox_geom = ee.Geometry.Polygon({"coords": bbox_coords, "proj": cali_albers, "evenOdd": false});